In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 30679518
paper_name = 'alfatah_arumugam_2019' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/41598_2018_35979_MOESM2_ESM.xlsx', sheet_name='HOP_Hypoculoside')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3702 x 7


In [7]:
original_data['genes'] = original_data['genes'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [9]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['genes'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])

In [11]:
print(original_data.loc[~t,])

             Unnamed: 0  genes     logFC    logCPM        LR  P-value  \
index_input                                                             
2078               2658  37165 -0.318685  6.774557  0.618299  0.43168   

                  FDR   orfs  
index_input                   
2078         0.768177  37165  


In [12]:
to_remove = original_data.loc[original_data['genes'] == '37165',:].index.values
original_data.drop(index=to_remove, inplace=True)

In [13]:
original_data.shape

(3701, 8)

In [14]:
# If the same strain is present more than once, average its values
data = original_data.groupby('orfs')['logFC'].mean().to_frame()

In [15]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 3683 x 1


In [17]:
data.index.name='orf'

# Prepare the final dataset

In [18]:
dataset_ids = [16440]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,16440
data_type,value
orf,
YAL002W,-5.242353
YAL004W,0.277069
YAL005C,0.215890
YAL007C,0.268790
YAL008W,-0.208429


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 18


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [23]:
data.head()

,dataset_id,16440
,data_type,value
gene_id,orf,
2,YAL002W,-5.242353
1863,YAL004W,0.277069
4,YAL005C,0.215890
5,YAL007C,0.268790
6,YAL008W,-0.208429


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [27]:
data_all.head()

dataset_id          16440          
data_type           value    valuez
gene_id orf                        
2       YAL002W -5.242353 -7.871263
1863    YAL004W  0.277069  0.320588
4       YAL005C  0.215890  0.229787
5       YAL007C  0.268790  0.308299
6       YAL008W -0.208429 -0.399983

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 30679518...
Inserting the new data...


100%|██████████| 1/1 [00:05<00:00,  5.43s/it]

Updating the data_modified_on field...
